In [2]:

class segurança:
    def __init__(self, name, tipo):
        self.name = name
        self.tipo = tipo
        
        
    def verif(self):
        if self.name == 'douglas':
            print(f'o nome do usuário {name} tem acesso')
            chave = True
        else:
            print(f'o usuário {name} não tem acesso')
            chave = False
        return chave
    
    def calculo(self, nm1, nm2):
        chave = self.verif()
        if chave == True:
            if tipo == 'soma':
                resultado = nm1 + nm2
                print(f'o resultado é {resultado}')
            elif tipo == 'subtração':
                resultado = nm1 - nm2
                print(f'o resultado é {resultado}')
            else:
                print('esse tipo não existe')
        elif chave == False:
            print('não pode calcular')

    def teste_nome(self):
        print(f'{self.name}')
        print(f'{self.tipo}')



In [ ]:
name = 'douglas'
tipo = 'subtração'

In [ ]:
exe = segurança(name, tipo)

exe.teste_nome()

douglas
subtração


In [ ]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Cria uma sessão do Spark
spark = SparkSession.builder.appName("PokemonData").getOrCreate()

# Função para obter uma lista de Pokémon com paginação
def get_pokemon_names(offset, limit):
    base_url = f"https://pokeapi.co/api/v2/pokemon?offset={offset}&limit={limit}"
    response = requests.get(base_url)
    
    if response.status_code == 200:
        data = response.json()
        return [pokemon['name'] for pokemon in data['results']]
    else:
        print(f"Error: Could not retrieve the list of Pokémon. Status code: {response.status_code}")
        return []

# Função para obter dados de um Pokémon específico e retornar como um dicionário
def get_pokemon_data(pokemon_name):
    base_url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name.lower()}"
    response = requests.get(base_url)
    
    if response.status_code == 200:
        data = response.json()
        return {
            "name": data['name'].capitalize(),
            "height": data['height'],
            "weight": data['weight'],
            "types": ", ".join([type_info['type']['name'].capitalize() for type_info in data['types']]),
            "abilities": ", ".join([ability['ability']['name'].capitalize() for ability in data['abilities']])
        }
    else:
        print(f"Error: Could not retrieve data for {pokemon_name}. Status code: {response.status_code}")
        return None

# Configurações de paginação
offset = 0
limit = 10000
total_pokemon = 0
pokemon_data = []

#função de paginação
while True:
    #1000 Pokémon começando do offset atual
    pokemon_names = get_pokemon_names(offset, limit)
    
    # Se a lista estiver vazia, terminamos a extração
    if not pokemon_names:
        break
    
    # Coleta os dados de todos os Pokémon nesse lote
    for pokemon in pokemon_names:
        data = get_pokemon_data(pokemon)
        if data:
            pokemon_data.append(Row(**data))
    
    # Atualiza offset para o próximo lote
    offset += limit
    total_pokemon += len(pokemon_names)
    print(f"Processed {total_pokemon} Pokémon so far...")

# Cria DataFrame a partir da lista de Rows
#função de criação de tabela
pokemon_df = spark.createDataFrame(pokemon_data)

display(pokemon_df)

Processed 1302 Pokémon so far...


name,height,weight,types,abilities
Bulbasaur,7,69,"Grass, Poison","Overgrow, Chlorophyll"
Ivysaur,10,130,"Grass, Poison","Overgrow, Chlorophyll"
Venusaur,20,1000,"Grass, Poison","Overgrow, Chlorophyll"
Charmander,6,85,Fire,"Blaze, Solar-power"
Charmeleon,11,190,Fire,"Blaze, Solar-power"
Charizard,17,905,"Fire, Flying","Blaze, Solar-power"
Squirtle,5,90,Water,"Torrent, Rain-dish"
Wartortle,10,225,Water,"Torrent, Rain-dish"
Blastoise,16,855,Water,"Torrent, Rain-dish"
Caterpie,3,29,Bug,"Shield-dust, Run-away"


In [ ]:
pokemon_df.createOrReplaceTempView("tb_pokemon")

In [ ]:
%sql
select 
  name as ds_name
  , height as nr_height
  , weight as weight
  , types as ds_types
  , abilities as ds_abilities

from tb_pokemon

ds_name,nr_height,weight,ds_types,ds_abilities
Bulbasaur,7,69,"Grass, Poison","Overgrow, Chlorophyll"
Ivysaur,10,130,"Grass, Poison","Overgrow, Chlorophyll"
Venusaur,20,1000,"Grass, Poison","Overgrow, Chlorophyll"
Charmander,6,85,Fire,"Blaze, Solar-power"
Charmeleon,11,190,Fire,"Blaze, Solar-power"
Charizard,17,905,"Fire, Flying","Blaze, Solar-power"
Squirtle,5,90,Water,"Torrent, Rain-dish"
Wartortle,10,225,Water,"Torrent, Rain-dish"
Blastoise,16,855,Water,"Torrent, Rain-dish"
Caterpie,3,29,Bug,"Shield-dust, Run-away"
